# 1. create positive / negative label

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler

In [46]:
df1=pd.read_csv(r"D:\CU\spring 2022\nlp\Roku\amazon_OS10.csv")
df2=pd.read_csv(r"D:\CU\spring 2022\nlp\Roku\amazon_OS94.csv")
df3=pd.read_csv(r"D:\CU\spring 2022\nlp\Roku\amazon_OS10_5.csv")

In [47]:
df=pd.concat([df1,df2,df3],axis=0)

In [48]:
df["label"]=[1 if i==5 or i==4 else 0 for i in df["rating"]]
# 1 is positive, 0 is negative

In [49]:
df["label"].value_counts()
# too less negative, might result in inaccurate model, over sampling needed later

1    1114
0     600
Name: label, dtype: int64

# 2. Vectorizing and oversample 

In [50]:
def my_bow(df_in, gram_m, gram_n, sw):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfVectorizer
    import pandas as pd
    if sw == "tf-idf":
        my_cv = TfidfVectorizer(ngram_range=(gram_m, gram_n))
    else:
        my_cv = CountVectorizer(ngram_range=(gram_m, gram_n))
    my_cv_data = pd.DataFrame(my_cv.fit_transform(df_in).toarray())
    col_names = list(my_cv.vocabulary_.keys())
    my_cv_data.columns = col_names
    return my_cv_data

def ml_helper(model,x,y,xt,yt,params,cv,sc):
    from sklearn.model_selection import GridSearchCV
    grid=GridSearchCV(model, param_grid=params, cv=cv, scoring=sc)
    grid.fit(x,y)
    print("best mean cv score {}".format(grid.best_score_))
    print("test score {}".format(grid.score(xt, yt)))
    print("best parameters {}".format(grid.best_params_))
    return None

In [51]:
df4=df[["stemmed","id","label"]]
x=df4[["stemmed","id"]]
y=df4[["label"]]

In [52]:
# over sample
oversample = RandomOverSampler(sampling_strategy='minority')
x_over,y=oversample.fit_resample(x, y)

In [53]:
df5=pd.concat([x_over,y],axis=1)

In [54]:
df6=df5.merge(df, how="left", on="id")

In [59]:
df6.columns

Index(['stemmed_x', 'id', 'label_x', 'title', 'body', 'link', 'rating',
       'verified_purchase', 'helpful_votes', 'year', 'month', 'version',
       'bd_cleaned', 'sw_removed', 'stemmed_y', 'label_y'],
      dtype='object')

In [64]:
df6.to_csv(r"D:\CU\spring 2022\nlp\Roku\amazon_model_df.csv")

In [60]:
# 这里搞出了两个可以用在后面traning的dataset，一个是bow一个是tfidf，后面看那个效果好就用那个
# 另外，中间的1，3代表我们看1个单词或者3个单词组成的词组的情况，可以视情况而定要不要进行调整（1，1），（1，2）
bow_df=my_bow(df6["stemmed_x"], 1, 3, "bow")
tfidf_df=my_bow(df6["stemmed_x"], 1, 3, "tf-idf")

In [62]:
y=df6["label_x"]